In [1]:
import os
from dotenv import load_dotenv
import sqlite3
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich import box

In [2]:
# Load environment variables
load_dotenv()

console = Console()

In [3]:
DB_FILE = os.getenv("DB_FILE", "game_database.db")
DB_PASSWORD = os.getenv("DB_PASSWORD")

In [4]:
def get_connection():
    conn = sqlite3.connect(DB_FILE, timeout=10)  # Wait up to 10 seconds
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA key = '{DB_PASSWORD}';")   # Your existing key setting
    cursor.execute("PRAGMA foreign_keys = ON;")        # Enable foreign key support
    cursor.close()
    return conn


In [5]:
def execute_query(query: str):
    try:
        conn = get_connection()
        cursor = conn.cursor()

        console.print("[bold cyan]📡 Executing Query...[/bold cyan]")
        cursor.execute(query)

        if query.strip().lower().startswith("select"):
            rows = cursor.fetchall()
            if rows:
                columns = [description[0] for description in cursor.description]

                table = Table(title="Query Results", box=box.ROUNDED, show_lines=True)
                for col in columns:
                    table.add_column(col, style="bold magenta")

                for row in rows:
                    table.add_row(*[str(cell) for cell in row])

                console.print(table)
                console.print(f"✅ [green]{len(rows)} rows fetched successfully.[/green]")
            else:
                console.print(Panel("[yellow]⚠️ Query returned no rows.[/yellow]", title="No Data", style="yellow"))
        else:
            conn.commit()
            console.print(Panel(f"✅ [green]Query executed successfully.[/green]", title="Success", style="green"))

    except Exception as e:
        console.print(Panel(f"❗ [red]Error executing query:[/red] {str(e)}", title="Error", style="red"))

    finally:
        cursor.close()
        conn.close()
        console.print("[bold cyan]🔌 Connection closed.[/bold cyan]\n")

In [6]:
def list_tables():
    """List all tables with row counts."""
    query = """
    SELECT name FROM sqlite_master 
    WHERE type='table' AND name NOT LIKE 'sqlite_%'
    ORDER BY name;
    """
    try:
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute(query)
        tables = cursor.fetchall()

        if not tables:
            console.print(Panel("[yellow]⚠️ No tables found in the database.[/yellow]", title="No Tables", style="yellow"))
            return

        table = Table(title="Tables in Database", box=box.ROUNDED, show_lines=True)
        table.add_column("Table Name", style="bold green")
        table.add_column("Number of Records", style="bold cyan", justify="right")

        for (table_name,) in tables:
            cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
            count = cursor.fetchone()[0]
            table.add_row(f"📋 {table_name}", str(count))

        console.print(table)
    except Exception as e:
        console.print(Panel(f"❗ [red]Error listing tables:[/red] {str(e)}", title="Error", style="red"))
    finally:
        cursor.close()
        conn.close()
        console.print("[bold cyan]🔌 Connection closed.[/bold cyan]\n")

In [7]:
def table_info(table_name: str):
    """Show info about table: columns, row count, and sample rows (max 10)."""
    try:
        conn = get_connection()
        cursor = conn.cursor()

        # Check if table exists
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
        if not cursor.fetchone():
            console.print(Panel(f"⚠️ Table '{table_name}' does not exist.", style="yellow"))
            return

        # Get columns info
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns_info = cursor.fetchall()
        columns = [col[1] for col in columns_info]

        # Get row count
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        row_count = cursor.fetchone()[0]

        # Print columns and row count
        info_panel = Panel.fit(
            f"📋 [bold green]Table:[/bold green] {table_name}\n"
            f"🔢 [bold cyan]Columns:[/bold cyan] {', '.join(columns)}\n"
            f"📊 [bold magenta]Total Rows:[/bold magenta] {row_count}",
            title="Table Info",
            style="blue",
        )
        console.print(info_panel)

        # Fetch and show up to 10 rows
        cursor.execute(f"SELECT * FROM {table_name} LIMIT 10")
        rows = cursor.fetchall()

        if rows:
            table = Table(title=f"Sample Data from {table_name}", box=box.ROUNDED, show_lines=True)
            for col in columns:
                table.add_column(col, style="bold magenta")
            for row in rows:
                table.add_row(*[str(cell) for cell in row])
            console.print(table)
        else:
            console.print(Panel("[yellow]⚠️ No data found in the table.[/yellow]", title="Empty Table", style="yellow"))

    except Exception as e:
        console.print(Panel(f"❗ [red]Error fetching table info:[/red] {str(e)}", title="Error", style="red"))
    finally:
        cursor.close()
        conn.close()
        console.print("[bold cyan]🔌 Connection closed.[/bold cyan]\n")

In [8]:
list_tables()

          Tables in Database           
╭─────────────────┬───────────────────╮
│ Table Name      │ Number of Records │
├─────────────────┼───────────────────┤
│ 📋 achievements │                 0 │
├─────────────────┼───────────────────┤
│ 📋 avatars      │                 0 │
├─────────────────┼───────────────────┤
│ 📋 leagues      │                13 │
├─────────────────┼───────────────────┤
│ 📋 legs         │                 0 │
├─────────────────┼───────────────────┤
│ 📋 matches      │                 4 │
├─────────────────┼───────────────────┤
│ 📋 players      │                 2 │
├─────────────────┼───────────────────┤
│ 📋 predictions  │                 0 │
├─────────────────┼───────────────────┤
│ 📋 rounds       │                 2 │
├─────────────────┼───────────────────┤
│ 📋 team_league  │                61 │
├─────────────────┼───────────────────┤
│ 📋 teams        │                23 │
╰─────────────────┴───────────────────╯

🔌 Connection closed.

In [11]:
def drop_tables_interactive():
    try:
        conn = get_connection()
        cursor = conn.cursor()

        # Get list of user tables
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';")
        tables = [row[0] for row in cursor.fetchall()]

        if not tables:
            print("No user tables found to drop.")
            return

        print("\n📋 Existing tables:")
        for idx, table in enumerate(tables, 1):
            print(f"{idx}. {table}")

        print("\nOptions:")
        print("  🔢 Enter the table number to drop a specific table")
        print("  💣 Enter 'all' to drop ALL tables")
        print("  ❌ Enter 'exit' to cancel and exit")

        while tables:
            choice = input("\nYour choice: ").strip().lower()

            if choice == 'exit':
                print("🚪 Exit without dropping tables.")
                break

            elif choice == 'all':
                for table in tables:
                    try:
                        cursor.execute(f"DROP TABLE IF EXISTS {table};")
                        print(f"✅ Dropped table '{table}'")
                    except sqlite3.OperationalError as e:
                        print(f"❌ Failed to drop '{table}': {e}")
                conn.commit()
                break

            elif choice.isdigit():
                idx = int(choice)
                if 1 <= idx <= len(tables):
                    table = tables[idx - 1]
                    try:
                        cursor.execute(f"DROP TABLE IF EXISTS {table};")
                        conn.commit()
                        print(f"✅ Dropped table '{table}'")
                        tables.pop(idx - 1)
                    except sqlite3.OperationalError as e:
                        print(f"❌ Failed to drop '{table}': {e}")

                    if not tables:
                        print("📭 No more tables left.")
                        break
                    else:
                        print("\n📋 Remaining tables:")
                        for i, t in enumerate(tables, 1):
                            print(f"{i}. {t}")
                else:
                    print("⚠️ Invalid table number, try again.")
            else:
                print("⚠️ Invalid input, try again.")

    except sqlite3.OperationalError as e:
        print(f"🚫 Database operation failed: {e}")
    finally:
        try:
            cursor.close()
        except:
            pass
        try:
            conn.close()
        except:
            pass


In [13]:
drop_tables_interactive()


📋 Existing tables:
1. rounds
2. teams
3. matches
4. legs
5. avatars
6. predictions
7. achievements
8. team_league
9. players
10. leagues

Options:
  🔢 Enter the table number to drop a specific table
  💣 Enter 'all' to drop ALL tables
  ❌ Enter 'exit' to cancel and exit
❌ Failed to drop 'teams': database is locked

📋 Remaining tables:
1. rounds
2. teams
3. matches
4. legs
5. avatars
6. predictions
7. achievements
8. team_league
9. players
10. leagues
🚪 Exit without dropping tables.


## 1.1 🧑‍💼 1. Players Table

In [ ]:
query_players = """
CREATE TABLE IF NOT EXISTS players (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT NOT NULL UNIQUE,
    email TEXT UNIQUE NOT NULL,
    password_hash TEXT NOT NULL,
    avatar_url TEXT,
    role TEXT CHECK(role IN ('admin', 'player')) DEFAULT 'player',
    is_confirmed INTEGER DEFAULT 1,
    reset_token TEXT,
    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
    updated_at TEXT DEFAULT CURRENT_TIMESTAMP,
    last_login_at TEXT DEFAULT NULL
);
"""
execute_query(query_players)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 2.2 🏆 2. Leagues Table

In [11]:
query_leagues = """
CREATE TABLE IF NOT EXISTS leagues (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE NOT NULL,
    country TEXT,
    logo_path TEXT,                          -- Path to local league logo
    can_be_draw INTEGER DEFAULT 1,           -- 1 = draw allowed
    two_legs INTEGER DEFAULT 0,              -- 1 = two-legged ties
    must_have_winner INTEGER DEFAULT 0,      -- 1 = no draw allowed (e.g., finals)
    created_at TEXT DEFAULT CURRENT_TIMESTAMP
);
"""
execute_query(query_leagues)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 3.3 📅 3. Rounds Table

In [12]:
query_rounds = """
CREATE TABLE IF NOT EXISTS rounds (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    start_date TEXT NOT NULL,
    end_date TEXT NOT NULL,
    created_at TEXT DEFAULT CURRENT_TIMESTAMP
);
"""
execute_query(query_rounds)

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 4.4 🏟️ 4. Teams Table

In [37]:
query_teams = """
CREATE TABLE IF NOT EXISTS teams (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL UNIQUE,
    logo_path TEXT,
    league_id INTEGER NOT NULL,
    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (league_id) REFERENCES leagues(id) ON DELETE CASCADE
);
"""
execute_query(query_teams)


📡 Executing Query...

╭───────────────────────────────────────────────────── Error ─────────────────────────────────────────────────────╮
│ ❗ Error executing query: database is locked                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

📡 Executing Query...

╭───────────────────────────────────────────────────── Error ─────────────────────────────────────────────────────╮
│ ❗ Error executing query: table teams already exists                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## team_league join table

In [ ]:
execute_query("""
CREATE TABLE IF NOT EXISTS team_league (
    team_id INTEGER NOT NULL,
    league_id INTEGER NOT NULL,
    PRIMARY KEY (team_id, league_id),
    FOREIGN KEY (team_id) REFERENCES teams(id) ON DELETE CASCADE,
    FOREIGN KEY (league_id) REFERENCES leagues(id) ON DELETE CASCADE
);
""")



📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 5.5 ⚽ 5. Matches Table

In [ ]:
query_matches = """
CREATE TABLE IF NOT EXISTS matches (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    round_id INTEGER NOT NULL,
    league_id INTEGER NOT NULL,
    home_team_id INTEGER NOT NULL,
    away_team_id INTEGER NOT NULL,
    match_datetime TEXT NOT NULL,
    status TEXT CHECK (status IN ('upcoming', 'live', 'finished', 'cancelled')) DEFAULT 'upcoming',
    home_score INTEGER DEFAULT NULL,
    away_score INTEGER DEFAULT NULL,
    FOREIGN KEY (round_id) REFERENCES rounds(id) ON DELETE CASCADE,
    FOREIGN KEY (league_id) REFERENCES leagues(id) ON DELETE CASCADE,
    FOREIGN KEY (home_team_id) REFERENCES teams(id) ON DELETE CASCADE,
    FOREIGN KEY (away_team_id) REFERENCES teams(id) ON DELETE CASCADE
);
"""
execute_query(query_matches)

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 6.6 🔁 6. Legs Table

In [16]:
query_legs = """
CREATE TABLE IF NOT EXISTS legs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    match_id INTEGER NOT NULL,
    leg_number INTEGER NOT NULL,
    leg_date TEXT NOT NULL,
    home_score INTEGER,
    away_score INTEGER,
    can_draw BOOLEAN DEFAULT 1,
    winner_team_id INTEGER,
    notes TEXT,
    FOREIGN KEY (match_id) REFERENCES matches(id) ON DELETE CASCADE,
    FOREIGN KEY (winner_team_id) REFERENCES teams(id) ON DELETE SET NULL,
    UNIQUE (match_id, leg_number)
);
"""
execute_query(query_legs)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 7.7 🖼️ 7. Avatars Table

In [17]:
query_avatars = """
CREATE TABLE IF NOT EXISTS avatars (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    file_path TEXT NOT NULL,                          -- Local path to the image
    uploaded_by_player_id INTEGER,                    -- Who uploaded the avatar
    uploaded_at TEXT DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (uploaded_by_player_id) REFERENCES players(id) ON DELETE SET NULL
);
"""
execute_query(query_avatars)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 8.8 predictions Table

In [18]:
query_predictions = """
CREATE TABLE IF NOT EXISTS predictions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    player_id INTEGER NOT NULL,
    match_id INTEGER NOT NULL,
    
    predicted_home_score INTEGER NOT NULL,
    predicted_away_score INTEGER NOT NULL,
    predicted_penalty_winner TEXT,                -- Team name or NULL
    
    score INTEGER DEFAULT 0,                      -- Computed after match ends

    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
    
    FOREIGN KEY (player_id) REFERENCES players(id) ON DELETE CASCADE,
    FOREIGN KEY (match_id) REFERENCES matches(id) ON DELETE CASCADE,
    
    UNIQUE (player_id, match_id)                  -- Player can only predict once per match
);
"""
execute_query(query_predictions)

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 9.9 achievements Table

In [19]:
query_achievements = """
CREATE TABLE IF NOT EXISTS achievements (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    player_id INTEGER NOT NULL,
    
    total_leagues_won INTEGER DEFAULT 0,
    total_cups_won INTEGER DEFAULT 0,

    updated_at TEXT DEFAULT CURRENT_TIMESTAMP,

    FOREIGN KEY (player_id) REFERENCES players(id) ON DELETE CASCADE,
    UNIQUE (player_id)
);
"""
execute_query(query_achievements)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

In [9]:
list_tables()

          Tables in Database           
╭─────────────────┬───────────────────╮
│ Table Name      │ Number of Records │
├─────────────────┼───────────────────┤
│ 📋 achievements │                 0 │
├─────────────────┼───────────────────┤
│ 📋 avatars      │                 0 │
├─────────────────┼───────────────────┤
│ 📋 leagues      │                11 │
├─────────────────┼───────────────────┤
│ 📋 legs         │                 0 │
├─────────────────┼───────────────────┤
│ 📋 matches      │                 0 │
├─────────────────┼───────────────────┤
│ 📋 players      │                 2 │
├─────────────────┼───────────────────┤
│ 📋 predictions  │                 0 │
├─────────────────┼───────────────────┤
│ 📋 rounds       │                 0 │
├─────────────────┼───────────────────┤
│ 📋 team_league  │                 0 │
├─────────────────┼───────────────────┤
│ 📋 teams        │                 0 │
╰─────────────────┴───────────────────╯

🔌 Connection closed.

In [47]:
table_info("team_league")

╭────────── Table Info ──────────╮
│ 📋 Table: team_league          │
│ 🔢 Columns: team_id, league_id │
│ 📊 Total Rows: 14              │
╰────────────────────────────────╯

   Sample Data from    
      team_league      
╭─────────┬───────────╮
│ team_id │ league_id │
├─────────┼───────────┤
│ 7       │ 3         │
├─────────┼───────────┤
│ 7       │ 5         │
├─────────┼───────────┤
│ 7       │ 6         │
├─────────┼───────────┤
│ 7       │ 9         │
├─────────┼───────────┤
│ 8       │ 3         │
├─────────┼───────────┤
│ 8       │ 5         │
├─────────┼───────────┤
│ 8       │ 7         │
├─────────┼───────────┤
│ 8       │ 9         │
├─────────┼───────────┤
│ 8       │ 6         │
├─────────┼───────────┤
│ 9       │ 3         │
╰─────────┴───────────╯

🔌 Connection closed.

In [87]:
table_info("matches")

╭────────────────────────────────────────────────── Table Info ───────────────────────────────────────────────────╮
│ 📋 Table: matches                                                                                               │
│ 🔢 Columns: id, round_id, league_id, home_team_id, away_team_id, match_datetime, status, home_score, away_score │
│ 📊 Total Rows: 0                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Empty Table ──────────────────────────────────────────────────╮
│ ⚠️ No data found in the table.                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

In [90]:
import sqlite3

conn = sqlite3.connect("game_database.db")
cur = conn.cursor()

try:
    cur.execute("ALTER TABLE players ADD COLUMN avatar_url TEXT")
except sqlite3.OperationalError as e:
    print("Column probably already exists:", e)

conn.commit()
conn.close()


In [41]:
def show_and_clear_teams():
    conn = get_connection()
    cursor = conn.cursor()

    # Fetch all records from teams
    cursor.execute("SELECT * FROM teams;")
    records = cursor.fetchall()
    print("Records before deletion:")
    if records:
        for row in records:
            print(row)
    else:
        print("No records found.")

    # Delete all records from teams
    cursor.execute("DELETE FROM teams;")
    conn.commit()

    # Fetch again to confirm deletion
    cursor.execute("SELECT * FROM teams;")
    records_after = cursor.fetchall()
    print("\nRecords after deletion:")
    if records_after:
        for row in records_after:
            print(row)
    else:
        print("No records found.")

    cursor.close()
    conn.close()


In [46]:
show_and_clear_teams()

Records before deletion:
(6, 'Barcelona', 'assets/leagues\\Barcelona.png', 3, '2025-06-06 15:59:32')

Records after deletion:
No records found.


In [45]:
def show_and_clear_team_league():
    conn = get_connection()
    cursor = conn.cursor()
    
    print("📋 Current records in 'team_league':")
    cursor.execute("SELECT * FROM team_league;")
    rows = cursor.fetchall()
    if rows:
        for row in rows:
            print(row)
    else:
        print("No records found.")
    
    # Delete all records
    cursor.execute("DELETE FROM team_league;")
    conn.commit()
    print("\n🗑️ Deleted all records from 'team_league'.")
    
    # Display again to confirm
    cursor.execute("SELECT * FROM team_league;")
    rows_after_delete = cursor.fetchall()
    print("\n📋 Records in 'team_league' after deletion:")
    if rows_after_delete:
        for row in rows_after_delete:
            print(row)
    else:
        print("No records found.")
    
    cursor.close()
    conn.close()

# Usage:
show_and_clear_team_league()

📋 Current records in 'team_league':
(3, 11)
(5, 3)
(5, 5)
(5, 6)
(5, 7)
(5, 9)
(6, 3)
(6, 5)
(6, 6)
(6, 9)

🗑️ Deleted all records from 'team_league'.

📋 Records in 'team_league' after deletion:
No records found.


In [56]:
def show_and_clear_tables():
    conn = get_connection()
    cursor = conn.cursor()

    # Fetch all table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [row[0] for row in cursor.fetchall()]

    if not tables:
        print("No tables found in the database.")
        cursor.close()
        conn.close()
        return

    print("Tables in the database:")
    for i, table in enumerate(tables, 1):
        print(f"{i}. {table}")

    # Ask user to select a table
    while True:
        try:
            choice = int(input("\nEnter the number of the table you want to inspect: "))
            if 1 <= choice <= len(tables):
                selected_table = tables[choice - 1]
                break
            else:
                print(f"Please enter a number between 1 and {len(tables)}.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

    # Show number of records in the selected table
    cursor.execute(f"SELECT COUNT(*) FROM {selected_table};")
    record_count = cursor.fetchone()[0]
    print(f"\nTable '{selected_table}' has {record_count} record(s).")

    # Ask if user wants to delete all records
    delete_confirm = input("Do you want to delete all records in this table? (yes/no): ").strip().lower()
    if delete_confirm == "yes":
        cursor.execute(f"DELETE FROM {selected_table};")
        conn.commit()
        print(f"All records in '{selected_table}' have been deleted.")

        # Confirm deletion
        cursor.execute(f"SELECT COUNT(*) FROM {selected_table};")
        new_count = cursor.fetchone()[0]
        print(f"Records remaining in '{selected_table}': {new_count}")
    else:
        print("No records were deleted.")

    cursor.close()
    conn.close()


In [105]:
show_and_clear_tables()

Tables in the database:
1. sqlite_sequence
2. rounds
3. teams
4. legs
5. avatars
6. predictions
7. achievements
8. team_league
9. players
10. leagues
11. matches

Table 'rounds' has 2 record(s).
All records in 'rounds' have been deleted.
Records remaining in 'rounds': 0


In [9]:
table_info("matches")
table_info("rounds")
table_info("teams")
table_info("team_league")

╭────────────────────────────────────────────────── Table Info ───────────────────────────────────────────────────╮
│ 📋 Table: matches                                                                                               │
│ 🔢 Columns: id, round_id, league_id, home_team_id, away_team_id, match_datetime, status, home_score, away_score │
│ 📊 Total Rows: 4                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                             Sample Data from matches                                              
╭────┬──────────┬───────────┬──────────────┬──────────────┬──────────────────┬──────────┬────────────┬────────────╮
│ id │ round_id │ league_id │ home_team_id │ away_team_id │ match_datetime   │ status   │ home_score │ away_score │
├────┼──────────┼───────────┼──────────────┼──────────────┼──────────────────┼──────────┼────────────┼────────────┤
│ 3  │ 7        │ 13        │ 29           │ 28           │ 2025-06-04       │ upcoming │ None       │ None       │
│    │          │           │              │              │ 22:15:00         │          │            │            │
├────┼──────────┼───────────┼──────────────┼──────────────┼──────────────────┼──────────┼────────────┼────────────┤
│ 4  │ 7        │ 13        │ 31           │ 29           │ 2025-06-05       │ upcoming │ None       │ None       │
│    │          │           │              │              │ 22:00:00         │          │            │            │
├────┼──────────┼───────────┼──────────────┼──────────────┼──────────────────┼──────────┼────────────┼────────────┤
│ 5  │ 8        │ 13        │ 29           │ 30           │ 2025-06-08       │ upcoming │ None       │ None       │
│    │          │           │              │              │ 16:00:00         │          │            │            │
├────┼──────────┼───────────┼──────────────┼──────────────┼──────────────────┼──────────┼────────────┼────────────┤
│ 6  │ 8        │ 13        │ 28           │ 31           │ 2025-06-08       │ upcoming │ None       │ None       │
│    │          │           │              │              │ 22:00:00         │          │            │            │
╰────┴──────────┴───────────┴──────────────┴──────────────┴──────────────────┴──────────┴────────────┴────────────╯

🔌 Connection closed.

╭────────────────────── Table Info ──────────────────────╮
│ 📋 Table: rounds                                       │
│ 🔢 Columns: id, name, start_date, end_date, created_at │
│ 📊 Total Rows: 2                                       │
╰────────────────────────────────────────────────────────╯

                    Sample Data from rounds                     
╭────┬─────────┬────────────┬────────────┬─────────────────────╮
│ id │ name    │ start_date │ end_date   │ created_at          │
├────┼─────────┼────────────┼────────────┼─────────────────────┤
│ 7  │ Round 1 │ 2025-05-31 │ 2025-06-06 │ 2025-06-07 14:11:45 │
├────┼─────────┼────────────┼────────────┼─────────────────────┤
│ 8  │ Round 2 │ 2025-06-07 │ 2025-06-13 │ 2025-06-07 14:12:54 │
╰────┴─────────┴────────────┴────────────┴─────────────────────╯

...

                                              Sample Data from teams                                               
╭────┬──────────────────┬───────────────────────────────────────────────────────┬───────────┬─────────────────────╮
│ id │ name             │ logo_path                                             │ league_id │ created_at          │
├────┼──────────────────┼───────────────────────────────────────────────────────┼───────────┼─────────────────────┤
│ 7  │ Barcelona        │ assets/leagues\Barcelona.png                          │ 3         │ 2025-06-06 16:03:35 │
├────┼──────────────────┼───────────────────────────────────────────────────────┼───────────┼─────────────────────┤
│ 8  │ Real Madrid      │ assets/leagues\86.png                                 │ 3         │ 2025-06-06 16:04:02 │
├────┼──────────────────┼───────────────────────────────────────────────────────┼───────────┼─────────────────────┤
│ 9  │ Atletico Madrid  │ assets/leagues\atletico-madrid-new-logo-png_seeklogo… │ 3         │ 2025-06-06 16:04:33 │
├────┼──────────────────┼───────────────────────────────────────────────────────┼───────────┼─────────────────────┤
│ 12 │ Athletic Club    │ assets/leagues\Athletic Club.png                      │ 3         │ 2025-06-06 23:36:12 │
├────┼──────────────────┼───────────────────────────────────────────────────────┼───────────┼─────────────────────┤
│ 13 │ Villarreal CF    │ assets/leagues\Villarreal CF.png                      │ 3         │ 2025-06-06 23:37:29 │
├────┼──────────────────┼───────────────────────────────────────────────────────┼───────────┼─────────────────────┤
│ 14 │ Real Betis       │ assets/leagues\RealBetis.png                          │ 3         │ 2025-06-06 23:41:05 │
├────┼──────────────────┼───────────────────────────────────────────────────────┼───────────┼─────────────────────┤
│ 15 │ RC Celta de Vigo │ assets/leagues\CeltaVigo.png                          │ 3         │ 2025-06-06 23:42:22 │
├────┼──────────────────┼───────────────────────────────────────────────────────┼───────────┼─────────────────────┤
│ 16 │ Rayo Vallecano   │ assets/leagues\RayoVallecano.png                      │ 3         │ 2025-06-06 23:45:36 │
├────┼──────────────────┼───────────────────────────────────────────────────────┼───────────┼─────────────────────┤
│ 17 │ CA Osasuna       │ assets/leagues\CA Osasuna.png                         │ 3         │ 2025-06-07 00:01:08 │
├────┼──────────────────┼───────────────────────────────────────────────────────┼───────────┼─────────────────────┤
│ 18 │ RCD Mallorca     │ assets/leagues\RCD Mallorca.png                       │ 3         │ 2025-06-07 00:03:19 │
╰────┴──────────────────┴───────────────────────────────────────────────────────┴───────────┴─────────────────────╯

🔌 Connection closed.

╭────────── Table Info ──────────╮
│ 📋 Table: team_league          │
│ 🔢 Columns: team_id, league_id │
│ 📊 Total Rows: 61              │
╰────────────────────────────────╯

   Sample Data from    
      team_league      
╭─────────┬───────────╮
│ team_id │ league_id │
├─────────┼───────────┤
│ 10      │ 6         │
├─────────┼───────────┤
│ 10      │ 17        │
├─────────┼───────────┤
│ 10      │ 7         │
├─────────┼───────────┤
│ 11      │ 6         │
├─────────┼───────────┤
│ 11      │ 17        │
├─────────┼───────────┤
│ 7       │ 3         │
├─────────┼───────────┤
│ 7       │ 5         │
├─────────┼───────────┤
│ 7       │ 6         │
├─────────┼───────────┤
│ 7       │ 9         │
├─────────┼───────────┤
│ 8       │ 3         │
╰─────────┴───────────╯

🔌 Connection closed.

In [101]:
conn = get_connection()
result = conn.execute("PRAGMA foreign_keys").fetchone()
print("Foreign keys enabled?", result[0])
conn.close()

Foreign keys enabled? 1


# Real Data

In [ ]:
import os
import requests

# Constants
API_TOKEN = '4c936cda044a4c2eaa3ed158b90e66d3'  # Replace with your actual API key
BASE_URL = 'https://api.football-data.org/v4'
HEADERS = {'X-Auth-Token': API_TOKEN}
LOGO_DOWNLOAD_PATH = r'D:\Football_Prediction_Game\assets\clubs\Team'

# Ensure the logo directory exists
os.makedirs(LOGO_DOWNLOAD_PATH, exist_ok=True)

def get_leagues():
    """Fetches a list of top-tier football leagues."""
    url = f'{BASE_URL}/competitions'
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        data = response.json()
        # Only return popular leagues
        popular_league_codes = ['PL', 'PD', 'BL1', 'SA', 'FL1', 'DED', 'PPL']
        leagues = [comp for comp in data['competitions'] if comp['code'] in popular_league_codes]
        return leagues
    else:
        print(f"Error fetching leagues: {response.status_code} - {response.text}")
        return []


def get_teams(league_id):
    """Fetches teams for a given league ID."""
    url = f'{BASE_URL}/competitions/{league_id}/teams?season=2024'
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        data = response.json()
        return data['teams']
    else:
        print(f"Error fetching teams: {response.status_code}")
        return []

def download_logo(team_name, logo_url):
    """Downloads the team's logo to the specified directory."""
    try:
        response = requests.get(logo_url, stream=True)
        if response.status_code == 200:
            # Sanitize team name for filename
            filename = f"{team_name.replace(' ', '_').replace('/', '_')}.png"
            filepath = os.path.join(LOGO_DOWNLOAD_PATH, filename)
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            print(f"Downloaded logo for {team_name}")
        else:
            print(f"Failed to download logo for {team_name}: {response.status_code}")
    except Exception as e:
        print(f"Error downloading logo for {team_name}: {e}")

def main():
    leagues = get_leagues()
    if not leagues:
        print("No leagues available.")
        return

    # Display leagues
    print("Available Leagues:")
    for idx, league in enumerate(leagues, start=1):
        print(f"{idx}. {league['name']} ({league['area']['name']})")

    # User selects a league
    try:
        choice = int(input("Select a league by number: "))
        if 1 <= choice <= len(leagues):
            selected_league = leagues[choice - 1]
            print(f"Fetching teams for {selected_league['name']}...")
            teams = get_teams(selected_league['id'])
            if teams:
                teams_dict = {}
                for team in teams:
                    team_name = team['name']
                    logo_url = team.get('crest')  # use 'crest' instead of 'crestUrl' in v4
                    teams_dict[team_name] = logo_url
                print("\nTeam Logos Dictionary:")
                for name, url in teams_dict.items():
                    print(f"{name}: {url}")
                return teams_dict
            else:
                print("No teams found for this league.")
        else:
            print("Invalid selection.")
    except ValueError:
        print("Please enter a valid number.")


if __name__ == "__main__":
    main()


Available Leagues:
1. Premier League (England)
2. Ligue 1 (France)
3. Bundesliga (Germany)
4. Serie A (Italy)
5. Eredivisie (Netherlands)
6. Primeira Liga (Portugal)
7. Primera Division (Spain)
Fetching teams for Premier League...

Team Logos Dictionary:
Arsenal FC: https://crests.football-data.org/57.png
Aston Villa FC: https://crests.football-data.org/58.png
Chelsea FC: https://crests.football-data.org/61.png
Everton FC: https://crests.football-data.org/62.png
Fulham FC: https://crests.football-data.org/63.png
Liverpool FC: https://crests.football-data.org/64.png
Manchester City FC: https://crests.football-data.org/65.png
Manchester United FC: https://crests.football-data.org/66.png
Newcastle United FC: https://crests.football-data.org/67.png
Tottenham Hotspur FC: https://crests.football-data.org/73.png
Wolverhampton Wanderers FC: https://crests.football-data.org/76.png
Leicester City FC: https://crests.football-data.org/338.png
Southampton FC: https://crests.football-data.org/340.pn

In [121]:
def main():
    leagues = get_leagues()
    if not leagues:
        print("No leagues available.")
        return

    # Display leagues
    print("Available Leagues:")
    for idx, league in enumerate(leagues, start=1):
        print(f"{idx}. {league['name']} ({league['area']['name']})")

    # User selects a league
    try:
        choice = int(input("Select a league by number: "))
        if 1 <= choice <= len(leagues):
            selected_league = leagues[choice - 1]
            league_name_clean = selected_league['name'].replace(' ', '_').replace('/', '_')
            print(f"\nFetching teams for {selected_league['name']}...\n")
            teams = get_teams(selected_league['id'])

            if teams:
                teams_dict = {}
                for team in teams:
                    team_name = team['name']
                    logo_url = team.get('crest')  # API v4 uses 'crest'
                    teams_dict[team_name] = logo_url
                    print(f"{team_name}: {logo_url}")

                # Save to text file
                file_path = os.path.join(LOGO_DOWNLOAD_PATH, f"{league_name_clean}.txt")
                with open(file_path, 'w', encoding='utf-8') as f:
                    for name, url in teams_dict.items():
                        f.write(f"{name}: {url}\n")

                print(f"\nSaved teams to {file_path}")
                return teams_dict
            else:
                print("No teams found for this league.")
        else:
            print("Invalid selection.")
    except ValueError:
        print("Please enter a valid number.")


if __name__ == "__main__":
    main()


Available Leagues:
1. Premier League (England)
2. Ligue 1 (France)
3. Bundesliga (Germany)
4. Serie A (Italy)
5. Eredivisie (Netherlands)
6. Primeira Liga (Portugal)
7. Primera Division (Spain)

Fetching teams for Ligue 1...

Toulouse FC: https://crests.football-data.org/511.png
Stade Brestois 29: https://crests.football-data.org/512.png
Olympique de Marseille: https://crests.football-data.org/516.png
Montpellier HSC: https://crests.football-data.org/518.png
AJ Auxerre: https://crests.football-data.org/519.png
Lille OSC: https://crests.football-data.org/521.png
OGC Nice: https://crests.football-data.org/522.png
Olympique Lyonnais: https://crests.football-data.org/523.png
Paris Saint-Germain FC: https://crests.football-data.org/524.png
AS Saint-Étienne: https://crests.football-data.org/527.png
Stade Rennais FC 1901: https://crests.football-data.org/529.png
Angers SCO: https://crests.football-data.org/532.png
Le Havre AC: https://crests.football-data.org/533.png
FC Nantes: https://crests

In [128]:
import sqlite3

def fetch_all(query, params=()):
    conn = get_connection()
    conn.row_factory = sqlite3.Row  # Enables dict-style row access
    cur = conn.cursor()
    cur.execute(query, params)
    rows = cur.fetchall()
    conn.close()
    return [dict(row) for row in rows]

def execute_query(query, params=()):
    conn = get_connection()
    cur = conn.cursor()
    cur.execute(query, params)
    conn.commit()
    conn.close()


In [133]:
def assign_team_to_leagues_console():
    # Fetch all teams
    teams = fetch_all("SELECT id, name FROM teams")
    if not teams:
        print("❌ No teams found.")
        return

    print("\n📋 List of Teams:")
    for i, team in enumerate(teams, start=1):
        print(f"{i}. {team['name']}")

    choice = int(input("\nEnter the number of the team to assign leagues: "))
    selected_team = teams[choice - 1]
    selected_team_id = selected_team['id']

    # Fetch all leagues
    leagues = fetch_all("SELECT id, name FROM leagues")
    if not leagues:
        print("❌ No leagues found.")
        return

    print("\n🏆 Available Leagues:")
    for i, league in enumerate(leagues, start=1):
        print(f"{i}. {league['name']}")

    selected_numbers = input("\nEnter the numbers of the leagues to assign (comma-separated): ")
    selected_indices = [int(x.strip()) for x in selected_numbers.split(",")]
    selected_league_ids = [leagues[i - 1]['id'] for i in selected_indices]

    # Delete old assignments
    execute_query("DELETE FROM team_league WHERE team_id = ?", (selected_team_id,))

    # Insert new assignments
    for league_id in selected_league_ids:
        execute_query(
            "INSERT INTO team_league (team_id, league_id) VALUES (?, ?)",
            (selected_team_id, league_id)
        )

    print(f"\n✅ Successfully updated leagues for team '{selected_team['name']}'.")

assign_team_to_leagues_console()


📋 List of Teams:
1. Athletic Club
2. Atletico Madrid
3. Barcelona
4. CA Osasuna
5. RC Celta de Vigo
6. RCD Mallorca
7. Rayo Vallecano
8. Real Betis
9. Real Madrid
10. Villarreal CF

🏆 Available Leagues:
1. Bundesliga
2. CAF Champions League
3. Club World Championship
4. Copa del Rey
5. Egyptian Premier League
6. LaLiga
7. Premier League
8. Seria A
9. Supercopa de España
10. UEFA Champions League
11. UEFA Conference League
12. UEFA Europa League
13. UEFA Nations League

✅ Successfully updated leagues for team 'Real Betis'.
